In [1]:
from langchain_openai import ChatOpenAI
from os import getenv
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from pageindex import PageIndexClient
import pageindex.utils as utils
import os

In [3]:
_pi_client = PageIndexClient(api_key=os.getenv("PAGEINDEX_API_KEY"))

In [4]:
import openai

async def call_llm(prompt, model="stepfun/step-3.5-flash:free", temperature=0):
    client = openai.AsyncOpenAI(api_key=os.getenv("OPENROUTER_API_KEY"))
    response = await client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

In [5]:
import requests

# You can also use our GitHub repo to generate PageIndex tree
# https://github.com/VectifyAI/PageIndex

pdf_url = "https://arxiv.org/pdf/2506.08276.pdf"
pdf_path = os.path.join("../data", pdf_url.split('/')[-1])
os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

response = requests.get(pdf_url)
with open(pdf_path, "wb") as f:
    f.write(response.content)
print(f"Downloaded {pdf_url}")

doc_id = _pi_client.submit_document(pdf_path)["doc_id"]
print('Document Submitted:', doc_id)

Downloaded https://arxiv.org/pdf/2506.08276.pdf
Document Submitted: pi-cmm2gj5it028p0ko9q2fyi5np


In [9]:
if _pi_client.is_retrieval_ready(doc_id):
    tree = _pi_client.get_tree(doc_id, node_summary=True)['result']
    print('Simplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

Simplified Tree Structure of the Document:
[{'title': 'ABSTRACT', 'node_id': '0000', 'summary': 'The text introduces LEANN, a storage-eff...'},
 {'title': '1 INTRODUCTION',
  'node_id': '0001',
  'summary': 'The text introduces vector search as a c...'},
 {'title': '3 LEANN Overview',
  'node_id': '0002',
  'summary': 'The text describes the LEANN system, whi...'},
 {'title': '4 GRAPH-BASED RECOMPUTATION',
  'node_id': '0003',
  'summary': '# 4 GRAPH-BASED RECOMPUTATION\n\nIn this s...'},
 {'title': '5 COMPACT GRAPH STRUCTURE',
  'node_id': '0004',
  'summary': "This text describes LEANN's approach to ..."},
 {'title': '6 Index Building and Update',
  'node_id': '0005',
  'summary': "The text describes LEANN's approach to b..."},
 {'title': '7 EVALUATION',
  'node_id': '0006',
  'prefix_summary': '# 7 EVALUATION\n\nWe begin by describing t...',
  'nodes': [{'title': '7.1 Experiment Settings',
             'node_id': '0007',
             'summary': 'The text details the experiment setti

In [10]:
import json

query = "What are the conclusions in this document?"

tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

tree_search_result = await call_llm(search_prompt)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-or-v1*************************************************************e8a6. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}